<a href="https://colab.research.google.com/github/Harshal292004/KGPDSH/blob/master/Copy_of_StormArchitecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel, Field, validator, PrivateAttr
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from google.colab import userdata
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import PathwayVectorClient
import os
from google.colab import userdata
import uuid  # To generate unique IDs
import json

client = PathwayVectorClient(url="https://demo-document-indexing.pathway.stream")

import json
import re
from typing import List, Dict
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

class ScoreDetails(BaseModel):
    score: float = Field(description="Score of the aspect (0-10)")
    justification: str = Field(description="Explanation for the score")

    @validator('score')
    def validate_score(cls, v: float) -> float:
        if v < 0 or v > 10:
            raise ValueError('Score must be between 0 and 10')
        return round(v, 1)

# Evaluation Categories
class Significance(BaseModel):
    theoretical_contribution: ScoreDetails = Field(description="Assessment of theoretical contribution to the field")
    practical_impact: ScoreDetails = Field(description="Evaluation of practical applications and real-world impact")
    novelty: ScoreDetails = Field(description="Originality of the research and its contribution")

class MethodologyEvaluation(BaseModel):
    research_design: ScoreDetails = Field(description="Appropriateness and quality of research design")
    reproducibility: ScoreDetails = Field(description="Clarity and completeness of methods for reproduction")

class PresentationQuality(BaseModel):
    writing_clarity: ScoreDetails = Field(description="Clarity and effectiveness of writing")
    organization: ScoreDetails = Field(description="Logical flow and structure of the paper")
    literature_review: ScoreDetails = Field(description="Comprehensiveness of literature review")

class PaperEvaluation(BaseModel):
    paper_id: str = Field(description="Unique identifier for the paper")
    significance: Significance = Field(description="Evaluation of research significance")
    methodology: MethodologyEvaluation = Field(description="Detailed methodology assessment")
    presentation: PresentationQuality = Field(description="Quality of presentation")
    confidence_level: float = Field(description="Overall confidence in this review (0-10)")
    _overall_score: float = PrivateAttr(default=0.0)
    major_strengths: List[str] = Field(description="Major strengths of the paper")
    major_weaknesses: List[str] = Field(description="Major weaknesses of the paper")
    detailed_feedback: str = Field(description="Comprehensive feedback and suggestions")
    summary_of_paper: str = Field(description="Relevant summary for retrieval tasks")

    @property
    def overall_score(self) -> float:
        if self._overall_score == 0.0:
            weights = {
                'significance': 0.3,
                'methodology': 0.5,
                'presentation': 0.2,
            }

            sig_score = (self.significance.theoretical_contribution.score +
                         self.significance.practical_impact.score +
                         self.significance.novelty.score) / 3

            meth_score = (self.methodology.research_design.score +
                          self.methodology.reproducibility.score) / 2

            pres_score = (self.presentation.writing_clarity.score +
                          self.presentation.organization.score +
                          self.presentation.literature_review.score) / 3

            self._overall_score = round(
                weights['significance'] * sig_score +
                weights['methodology'] * meth_score +
                weights['presentation'] * pres_score ,
                3
            )
        return self._overall_score

    @validator('confidence_level')
    def validate_confidence(cls, v: float) -> float:
        if v < 0 or v > 10:
            raise ValueError('Confidence level must be between 0 and 10')
        return round(v, 1)

    def generate_summary_report(self) -> str:
        string = f"""
        Paper Review Summary (ID: {self.paper_id})
        ==========================================
        Review Confidence: {self.confidence_level}/10
        Overall Score: {self.overall_score}/10
        Key Strengths:
        {chr(10).join(f"- {strength}" for strength in self.major_strengths)}

        Areas for Improvement:
        {chr(10).join(f"- {weakness}" for weakness in self.major_weaknesses)}

        Detailed Feedback:
        {self.detailed_feedback}

        Summary of the Paper:
        {self.summary_of_paper}
        """
        return string

# Implementing STORM Architecture with LangGraph
class ConferenceAgent:
    def __init__(self, name: str, chat_model: ChatGroq, target_conference: str, conference_themes: str, conference_context: str, vectorstore:PathwayVectorClient):
        self.name = name
        self.chat_model = chat_model
        self.target_conference = target_conference
        self.conference_themes = conference_themes
        self.conference_context = conference_context
        self.vector_store = vectorstore

    def retrieve_conference_context(self, paper_summary: str) -> str:
        results = self.vector_store.similarity_search(query=paper_summary, top_k=3)
        # Filter results for relevance to the conference
        return "\n".join([result.page_content for result in results if self.target_conference in result.page_content])

    def evaluate_paper(self, paper_summary: str) -> Dict:
        prompt = ChatPromptTemplate.from_messages([
            ('system',  f"""
                You are the representative for the conference {self.target_conference}.
                Theme: {self.conference_themes}
                Context: {self.conference_context}

                Evaluate the research significance and strengths.
                Score it (0-10) and justify why it fits or doesn't for the conference .
                Output must follow this JSON schema:
                - score: float
                - justification: str
            """),
            ('user', "Paper summary: {paper_summary}")
        ])
        chain = LLMChain(llm = self.chat_model, prompt=prompt)
        response = chain.run({"paper_summary": paper_summary})
        try:
              cleaned_response = re.sub(r'[\x00-\x1F\x7F]', '', response)
              json_match = re.search(r'\{.*\}', cleaned_response, re.DOTALL)
              if json_match:
                  json_response = json_match.group(0)
                  response_dict = json.loads(json_response)
                  return {
                      "score": float(response_dict["score"]),
                      "justification": response_dict["justification"],
                  }
              else:
                  # Fallback for non-JSON responses
                  return {"score": None, "justification": cleaned_response.strip()}
        except json.JSONDecodeError as e:
              raise ValueError(f"Failed to parse response as JSON: {response}") from e



class STORMSystem:
    def __init__(self, agents: List[ConferenceAgent], vectorstore: PathwayVectorClient):
        self.agents = agents
        self.vector_store = vectorstore

    def retrieve_context(self, paper_summary: str, target_conference: str) -> List[Dict]:
    # Adjust for the correct argument or truncate results manually
      try:
          results = self.vector_store.similarity_search(query=paper_summary)  # Remove `top_k`
        # Limit results manually
          results = results[:3] if len(results) > 3 else results
          return [result for result in results if target_conference in result.page_content]
      except Exception as e:
          print(f"Error during similarity search: {e}")
          return []

    def discuss_and_decide(self, paper_summary: str) -> Dict:
      evaluations = []

    # Loop through each agent to perform context retrieval and paper evaluation
      for agent in self.agents:
            try:
                # Retrieve conference-specific context
                relevant_context = agent.retrieve_conference_context(paper_summary)

                # Combine paper summary with retrieved context
                augmented_summary = f"{paper_summary}\n\nRelated Contexts:\n{relevant_context}"

                # Evaluate the paper using the agent
                evaluation = agent.evaluate_paper(augmented_summary)

                # Append the evaluation to the list
                evaluations.append({
                    "conference": agent.target_conference,
                    "score": evaluation.get("score", 0),
                    "justification": evaluation.get("justification", "No justification provided.")
                })

            except Exception as e:
                # Handle any errors during context retrieval or evaluation
                print(f"Error evaluating paper for {agent.target_conference}: {e}")
                evaluations.append({
                    "conference": agent.target_conference,
                    "score": 0,
                    "justification": f"Failed to evaluate paper due to: {e}"
                })

        # Determine the best conference based on scores

      best_conference = max(evaluations, key=lambda x: x["score"]  if x["score"] is not None else -1)

        # Store evaluations in the vector store for future reference
      for evaluation in evaluations:
            self.store_evaluation(paper_summary, evaluation)

        # Return the best conference and justification
      return {
            "best_conference": best_conference["conference"],
            "justification": best_conference["justification"]
        }

    def store_evaluation(self, paper_summary: str, evaluation: Dict):
        """Store evaluations in a local JSON file if vector store doesn't support additions."""
        metadata = {
            "best_conference": evaluation.get("conference", "Unknown"),
            "justification": evaluation.get("justification", "No justification provided."),
            "score": evaluation.get("score", 0),
        }
        try:
            # Save evaluations locally
            with open("evaluations.json", "a") as f:
                json.dump({"text": paper_summary, "metadata": metadata}, f)
                f.write("\n")
            print("Evaluation stored locally.")
        except Exception as e:
            print(f"Error storing evaluation locally: {e}")

# Example Usage
if __name__ == "__main__":
    chat_model = ChatGroq(groq_api_key = userdata.get("GROQ_API_KEY").strip(), model="llama-3.1-8b-instant")
    # Define agents for each conference
    agents = [
        ConferenceAgent(name="Agent CVPR", chat_model=chat_model, target_conference="CVPR",
                        conference_themes = """
                        CVPR focuses on the field of computer vision, image processing, and pattern recognition. Key themes include:
                          Object detection and recognition.
                          Image segmentation and scene understanding.
                          Visual tracking and motion analysis.
                          3D vision, stereo vision, and depth estimation.
                          Deep learning for vision tasks (CNNs, Vision Transformers, etc.).
                          Applications in medical imaging, autonomous vehicles, and robotics.
                          Video processing and understanding.
                          Computational photography and imaging.
                          Low-level vision (denoising, super-resolution).
                          Vision-based augmented reality and virtual reality.
                        """,
                        conference_context = """
                        Examples of accepted papers:
                          - YOLO: Real-Time Object Detection
                              Summary: Proposes an efficient deep learning model for real-time object detection with state-of-the-art accuracy.
                          - SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation
                              Summary: Focuses on pixel-wise image segmentation using deep neural networks, with applications in autonomous systems.
                          - 3D Object Detection and Localization Using RGB and Depth Data
                              Summary: Combines RGB and depth information for accurate 3D object detection in indoor environments.
                        Evaluation Criteria:
                         - Relevance to core computer vision topics.
                         - Advances in methodology or application.
                         - Practical implications for industries like healthcare, robotics, and transportation.
                        """, vectorstore=client
                        ),
        ConferenceAgent(name="Agent NeurIPS", chat_model=chat_model, target_conference="NeurIPS"
                        ,conference_themes ="""
                        NeurIPS emphasizes machine learning, computational neuroscience, and AI-related topics. Key themes include:
                            Deep learning (e.g., architectures, generative models, optimization for deep networks, foundation models, LLMs)
                            Reinforcement learning and decision-making.
                            Probabilistic models and Bayesian learning.
                            Optimization techniques for machine learning.
                            Representation learning and embeddings.
                            AI for healthcare, climate science, and sustainability.
                            Neuroscience-inspired algorithms and theories.
                            Ethical AI, fairness, and explainability.
                        """,
                        conference_context = """
                        Examples of accepted papers:
                          - Attention Is All You Need
                             Summary: Introduces the transformer architecture, which revolutionized NLP and other domains by using self-attention mechanisms.
                          - Generative Adversarial Networks (GANs)
                             Summary: Proposes a novel framework for training generative models using adversarial networks.
                          - Adam: A Method for Stochastic Optimization
                             Summary: Proposes a new optimization algorithm that balances convergence speed and stability in deep learning.
                        Evaluation Criteria:
                          - Novelty and theoretical contributions.
                          - Experimental rigor and reproducibility.
                          - Broader implications for machine learning and interdisciplinary applications.
                        """,
                        vectorstore=client
        ),
        ConferenceAgent(name="Agent EMNLP", chat_model=chat_model, target_conference="EMNLP"
                        ,conference_themes = """
                        EMNLP specializes in NLP and computational linguistics. Key themes include:
                          Machine translation and cross-lingual NLP.
                          Large language models and foundational models.
                          Sentiment analysis and opinion mining.
                          Dialogue systems and conversational AI.
                          Question answering and information retrieval.
                          Text summarization and abstraction.
                          Morphology, syntax, and semantics.
                          Multimodal NLP (text + image/audio fusion).
                          Ethical concerns in NLP (bias, toxicity detection).
                          Low-resource and multilingual NLP.
                        """,
                        conference_context = """
                        Examples of accepted papers:

Paper Title: Pre-trained Language Models for Text-to-Text Generation
Summary: This paper explores pre-trained models (e.g., T5) for a variety of text-to-text NLP tasks, showcasing improvements across summarization, question answering, and translation.

Paper Title: Adversarial Training for Robust Text Classification
Summary: Proposes an adversarial training framework that enhances the robustness of text classification models against noisy and adversarial inputs.

Paper Title: Knowledge-Enhanced Contextual Representations for Entity Linking
Summary: Combines external knowledge sources with contextual embeddings to improve entity linking in complex domains.

Paper Title: Multimodal Sentiment Analysis with Attention Mechanisms
Summary: Introduces a framework for combining visual and textual inputs to improve sentiment detection in videos.

Evaluation Criteria:
Relevance to NLP:

Papers should directly address core NLP tasks, methodologies, or applications.
Emphasis on empirical evaluation and innovation in language processing methods.
Quality of Methodology:

Rigorous experiments, proper baselines, and thorough ablation studies.
Use of diverse and large-scale datasets to validate results.
Novelty of Contribution:

New architectures, algorithms, or findings that advance the field.
Extensions of existing methods to novel tasks or domains.
Broader Impacts:

Ethical implications of the research (e.g., bias in models, data privacy).
Potential for cross-disciplinary applications (e.g., in healthcare, education, or social media).
Themes Breakdown with Examples:
Natural Language Understanding:

Semantic role labeling, coreference resolution, and discourse parsing.
Example: A New Framework for Semantic Parsing Using Pre-Trained Transformers.
Language Generation:

Dialogue systems, machine translation, and creative text generation.
Example: Controlled Text Generation with Discrete and Continuous Latent Variables.
Information Extraction:

Named entity recognition, relation extraction, and knowledge graph construction.
Example: Joint Entity and Relation Extraction with Transformer-Based Models.
Multimodal NLP:

Integrating vision, audio, and textual modalities.
Example: Aligning Text and Vision for Multimodal Machine Translation.
Social and Ethical Considerations:

Bias, fairness, interpretability, and environmental impacts of NLP models.
Example: Bias Mitigation in Pre-trained Models: A Data Augmentation Approach.

                        """,
                        vectorstore=client

        ),
        ConferenceAgent(name="Agent KDD", chat_model=chat_model, target_conference="KDD"
                        ,conference_themes = """
                        KDD centers on data mining, big data, and applied AI. Key themes include:
                            Scalable data mining algorithms.
                            Graph data and network analysis.
                            Temporal and sequential data mining.
                            Anomaly detection and predictive modeling.
                            Recommender systems and personalization.
                            Causal inference and counterfactual reasoning.
                            Applications in finance, e-commerce, and marketing.
                            Data visualization and interpretability.
                            Data ethics and privacy-preserving techniques.
                            AI and data-driven solutions for social good.
                        """,

                        conference_context="""
                        Examples of accepted papers:

Paper Title: Scalable Graph Neural Networks for Large-Scale Social Network Analysis
Summary: Introduces a scalable GNN framework that handles billion-scale graphs with high efficiency, showcasing applications in social network analysis.

Paper Title: Causal Discovery in High-Dimensional Data Using Deep Learning
Summary: Proposes a deep learning-based approach for identifying causal relationships in high-dimensional data.

Paper Title: Fair Representation Learning with Adversarial Networks
Summary: Develops a framework for learning fair data representations while minimizing demographic bias in downstream tasks.

Paper Title: Adaptive Online Learning for Real-Time Recommender Systems
Summary: Presents an adaptive online learning algorithm that updates recommender system models in real time based on user interactions.

Evaluation Criteria:
Relevance to Knowledge Discovery and Data Mining:

Papers must address core challenges in data mining, machine learning, or related applications.
Emphasis on practical applications and scalability to real-world data.
Methodological Rigor:

Strong theoretical foundations or innovative empirical methodologies.
Comprehensive experiments with real-world datasets and benchmarks.
Scalability and Efficiency:

Solutions must handle large-scale data effectively, both in computation and memory.
Novelty and Impact:

New techniques, algorithms, or applications that significantly advance the state-of-the-art.
Real-world relevance and potential societal or economic impact.
Broader Implications:

Ethical considerations, such as fairness, transparency, and responsible use of data.
Long-term applicability across industries and domains.
                        """,
                        vectorstore=client
        ),
    ]

    storm_system = STORMSystem(agents=agents,vectorstore=client)

    # Input a research paper summary
    paper_summary = """

output=PaperEvaluation(significance="Here are five detailed points summarizing the significance of the research:\n\n1. Addressing Plasticity Loss in Deep Reinforcement Learning: The research aims to tackle a critical issue in deep reinforcement learning, plasticity loss, which results in a decline in an agent's performance over time. By proposing a novel approach called 'plasticity injection,' the study has the potential to improve the long-term performance and adaptability of RL agents in dynamic environments.\n\n2. Improving Robustness and Reliability: The proposed method, plasticity injection, offers a diagnostic framework and a mitigation strategy to maintain consistent performance over extended training periods without incurring significant computational demands. This contribution has the potential to improve the robustness and reliability of RL agents in complex and dynamic environments.\n\n3. Enhancing Efficiency and Effectiveness: The targeted approach to selectively modify the network's internal dynamics without altering its overall architecture could improve the efficiency and effectiveness of RL agents in dynamic environments. Additionally, the use of a diagnostic framework to trigger resource allocation adds a layer of efficiency and adaptability to the method.\n\n4. Impact on Real-World Applications: The research has the potential to impact various areas of artificial intelligence, including robotics, computer vision, and natural language processing, where RL agents are increasingly being deployed. The proposed method could significantly improve the performance and adaptability of intelligent systems, making it a valuable contribution to the field.\n\n5. Potential to Advance State-of-the-Art in RL: The novel approach to dynamically adjusting artificial neural networks to maintain learning capacity over time could advance the state-of-the-art in reinforcement learning, particularly in complex real-world scenarios. The modular design of the framework allows for easy integration with various RL algorithms and architectures, making it a versatile tool for a wide range of applications. Here are the 5 detailed points summarizing the significance:\n\n1. Potential Impact on Artificial General Intelligence: This research has the potential to impact the field of artificial general intelligence, particularly in addressing the challenges of catastrophic forgetting. The novel contribution of dynamic capacity allocation in Reinforcement Learning (RL) agents with plasticity injection can enable agents to adapt to changing environments and tasks, advancing the state-of-the-art in RL.\n\n2. Improving Long-Term Performance and Learning Stability: The proposed approach aims to mitigate plasticity loss and improve the long-term performance and learning stability of RL agents. This is achieved through the injection of plasticity, which allows agents to adapt to changing environments and tasks, leading to improved performance and computational efficiency.\n\n3. Value in Practical Application and Field of RL: The paper's significance lies in its practical application and potential impact on the field of RL. The framework's modularity and computational efficiency enable it to adapt to different RL tasks and environments, achieving consistent improvement in long-term performance and learning stability.\n\n4. Contribution to Understanding Lifelong Learning in AI: This research explores the impact of plasticity injection on artificial agents, contributing to the understanding of lifelong learning in artificial intelligence. The study sheds light on the importance of individual components of the plasticity injection framework, providing a deeper understanding of their interplay.\n\n5. Potential to Benefit Broader Field of Reinforcement Learning: The paper's findings have the potential to benefit the broader field of reinforcement learning, as the proposed method is applicable to a diverse set of challenging RL environments, including continuous control tasks and partially observable Markov decision processes (POMDPs).


The results can also be used to inform the design of more efficient and effective plasticity injection frameworks.", methodology="1. Research Question and Objectives: The researchers aim to address the problem of plasticity loss in deep neural networks, with the objective of proposing a novel intervention called 'plasticity injection' to tackle this issue.\n\n2. Methodology: The proposed plasticity injection method involves a dynamic expansion mechanism triggered by a diagnostic framework, which is designed to selectively modify the network's internal dynamics and maintain computational efficiency while preventing performance degradation.\n\n3. Experimental Design: The researchers employed a rigorous experimental setup, including the use of RL benchmarks and comparison with state-of-the-art baselines, to evaluate the effectiveness of the plasticity injection framework. The evaluation metrics used include long-term performance, learning stability, and computational efficiency.\n\n4. Evaluation and Comparison: The researchers conducted ablation studies to systematically analyze the contributions of individual components to overall performance improvement. The study also employed statistical tests, including paired t-tests, to determine the statistical significance of performance differences between agents with and without plasticity injection.\n\n5. Contributions and Limitations: The proposed plasticity injection method is a novel approach to addressing plasticity loss in deep neural networks, and the study provides a comprehensive evaluation of its effectiveness across diverse RL environments. However, the paper could benefit from more detailed explanations of the diagnostic framework, mitigation strategy, and experimental design, as well as a more detailed discussion of the experimental results and the choice of evaluation metrics.", presentation="1. Improvement of Introduction and Related Work Section: To enhance the reader's understanding and appreciation of the research, it is recommended to provide more detailed background information on the problem and existing approaches. This can include a more comprehensive introduction, a stronger connection to related work, and additional context to facilitate understanding.\n\n2. Increased Use of Visual Aids: To make the paper more engaging and easier to understand, it is suggested to incorporate more visual aids, such as diagrams, flowcharts, tables, and figures, to illustrate the proposed approach, its components, and the results.\n\n3. Enhanced Clarity and Conciseness: To improve the paper's clarity and conciseness, it is recommended to use simpler language, provide more concrete examples, and avoid overly formal or technical jargon. Additionally, providing more detailed explanations and examples in sections such as the related work and methodology can facilitate understanding.\n\n4. More Context and Background Information: To make the paper more accessible to readers unfamiliar with the topic, it is suggested to provide more context and background information on the problem domain and the plasticity injection framework. This can include a more detailed introduction to the background and context of the research.\n\n5. Increased Use of Specific Examples and Visual Aids: To help illustrate the key points, it is recommended to incorporate more specific examples and visual aids, such as figures and tables, to make the text more engaging and easier to understand. This can also help to facilitate understanding of the results and the proposed approach.", confidence_score=7.774193548387097, score=7.53548387096774, major_strengths="Here are the 5 detailed points summarizing the major strengths of the research:\n\n1. Novel and Effective Approach to Plasticity Loss: The research proposes a novel and efficient approach, 'plasticity injection,' to address a significant problem in deep reinforcement learning, offering a proactive diagnostic framework and targeted intervention to prevent plasticity loss in RL agents.\n\n2.

Preservation of Prediction Capabilities and Selective Modification of Network Dynamics: The approach successfully maintains consistent performance without incurring significant computational demands, and selectively modifies the network's internal dynamics rather than its overall architecture, ensuring that the prediction capabilities of the network are preserved.\n\n3. Comprehensive Evaluation and Comparison with Existing Approaches: The research presents a thorough evaluation and comparison with existing approaches, using well-established evaluation metrics, and demonstrates the effectiveness of the proposed method in mitigating plasticity loss through comprehensive experimental results.\n\n4. Modular Design, Computational Efficiency, and Adaptability: The proposed plasticity injection framework is designed to be modular, allowing for easy integration with various RL algorithms and architectures, and is computationally efficient and scalable, making it a valuable tool for enhancing the robustness and longevity of RL agents.\n\n5. Clear Presentation, Logical Flow of Ideas, and Practical Implications: The paper presents a clear and well-organized argument, making it easy to follow and understand, and highlights the practical implications of the research, including improved adaptability and applicability of RL agents, and the potential to benefit the broader field of reinforcement learning.", major_weaknesses="Here are the 5 major weaknesses of the paper:\n\n1. Lack of detailed explanations and empirical evaluations: The paper lacks detailed explanations of the proposed method, its underlying mechanisms, and its efficiency and scalability. Additionally, the evaluation of the proposed approach is limited to a single experiment, which may not be sufficient to demonstrate its effectiveness.\n\n2. Insufficient connections to existing research and limited scope: The paper could benefit from more explicit connections to existing research in the field, a stronger introduction that provides specific details on the methodology and background information, and a more comprehensive evaluation of its generalizability.\n\n3. Limited discussion of potential limitations and challenges: The paper does not discuss the potential limitations and challenges of the proposed approach, which may hinder its applicability to real-world scenarios.\n\n4. Lack of detailed implementation and evaluation details: The paper lacks detailed implementation and evaluation details, making it difficult to fully assess the proposed approach. Additionally, the reliance on existing RL environments and frameworks limits the novelty of the contribution.\n\n5. Limited novelty and scope of evaluation metrics: The paper's novelty is somewhat limited by its reliance on existing techniques, and the evaluation metrics used may not capture the full range of potential benefits and drawbacks of the plasticity injection method. The paper could benefit from more comprehensive evaluation and comparison with existing methods, as well as a more detailed description of the experimental design and parameters.", justification="Here are the detailed points for the justification string:\n\n1. Clear and concise writing style: The paper's writing style is clear and concise, making it easy to understand and follow. This is a significant strength, as it allows readers to quickly grasp the main points of the research.\n\n2. Potential impact on the field: The paper has the potential to make a significant contribution to the field of Reinforcement Learning (RL), with its novel approach to addressing plasticity loss. This potential impact is a major factor in the overall score.\n\n3. Lack of detailed explanations and empirical evaluations: Despite the paper's potential impact, there is a lack of detailed explanations and empirical evaluations to further validate the effectiveness of the proposed method. This is a significant weakness, as it reduces the research's rigor and reproducibility.\n\n4.

Well-defined methodology and presentation: The paper's methodology is well-defined, and the presentation is clear and well-organized. This is a major strength, as it allows readers to easily follow the research and understand the results.\n\n5. Need for further refinement and detail: While the paper shows promise, it could benefit from additional details on the experimental setup, theoretical underpinnings, and practical applications to further strengthen its impact and reproducibility.", detailed_feedback="1. Enhance Introduction and Methodology: The researchers should provide a more detailed introduction explaining the significance and relevance of the research, proposed method, and its differences from existing approaches. A clear and concise methodology section should be included to provide a better understanding of the research.\n\n2. Provide Context and Background Information: The authors should provide more context on the problem and existing approaches, describe the proposed intervention in detail, and discuss the expected efficiency and adaptability of their solution. A background information section on the research question, literature review, and theoretical framework would enhance the paper's significance and impact.\n\n3. Validate Novel Metrics and Approach: The authors should provide more details on the derivation and validation of novel metrics used to capture subtle changes in network behavior. A more comprehensive evaluation of the proposed approach, including multiple experiments and comparisons to state-of-the-art methods, would be beneficial.\n\n4. Discuss Potential Limitations and Future Directions: The authors should discuss the potential limitations of their approach and potential areas for future research. They could also explore the integration of plasticity injection with other RL algorithms and architectures to further demonstrate its versatility and potential.\n\n5. Improve Writing Style and Reproducibility: The paper would benefit from a more detailed discussion of the potential real-world applications and limitations of the approach. The authors should provide more details on the experimental design, sample size, and agent architectures to enhance the reproducibility of the results.", publishable=True)
"""
    decision = storm_system.discuss_and_decide(paper_summary)
    print(f"Best Conference: {decision['best_conference']}")
    print(f"Justification: {decision['justification']}")



<ipython-input-7-da90471dcdba>:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('score')
<ipython-input-7-da90471dcdba>:87: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('confidence_level')


Evaluation stored locally.
Evaluation stored locally.
Evaluation stored locally.
Evaluation stored locally.
Best Conference: NeurIPS
Justification: The research has the potential to make a significant contribution to the field of Reinforcement Learning (RL) by addressing the critical issue of plasticity loss, which results in a decline in an agent's performance over time. The proposed approach, plasticity injection, is novel and effective, and the framework's modularity, computational efficiency, and adaptability make it a valuable tool for enhancing the robustness and longevity of RL agents. However, the paper could benefit from more detailed explanations, empirical evaluations, and a more comprehensive evaluation of its generalizability, novelty, and practical implications.


In [1]:
pip install langchain langchain_community langgraph langchain_groq langchain_huggingface pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install --upgrade langgraph

In [ ]:
# Pathway connector
from langchain_community.vectorstores import PathwayVectorClient

client = PathwayVectorClient(url="https://demo-document-indexing.pathway.stream")


In [ ]:
query = "What is Pathway?"
docs = client.similarity_search(query, timeout=0)

In [ ]:
docs[0].page_content

'3 2 0 2\nl u J\n2 1\n]\nG L . s c [\n1 v 6 1 1 3 1 . 7 0 3 2 : v i X r a\nPathway: a fast and flexible unified stream data processing framework for analytical and Machine Learning applications\nMichał Bartoszkiewicz\nJan Chorowski∗\nAdrian Kosowski\nJakub Kowalski\nSergey Kulik\nMateusz Lewandowski\nKrzysztof Nowicki\nKamil Piechowiak\nOlivier Ruas\nZuzanna Stamirowska\nPrzemysław Uznański\n{firstname.lastname}@pathway.com Pathway.com Paris, France\nABSTRACT We present Pathway, a new unified data processing framework that can run workloads on both bounded and unbounded data streams. The framework was created with the original motivation of re- solving challenges faced when analyzing and processing data from the physical economy, including streams of data generated by IoT and enterprise systems. These required rapid reaction while calling for the application of advanced computation paradigms (machine- learning-powered analytics, contextual analysis, and other elements of complex event 

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pathway.xpacks.llm.vector_store import VectorStoreServer
import os

os.environ['HF_TOKEN'] = userdata.get("HF_TOKEN")

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


import pathway as pw

# Example data for table creation
data = [
    {"text": "This is a test justification.", "id": "1"},
    {"text": "Another piece of text.", "id": "2"}
]

try:
    # Create a Pathway table from records
    pathway_table = pw.Table.from_records(
        data,
        schema=pw.Schema(columns={
            "text": pw.Column(type=str),
            "id": pw.Column(type=str)
        })
    )
    print("Pathway Table created successfully!")
except Exception as e:
    print(f"Error creating Pathway Table: {e}")

Error creating Pathway Table: 'super' object has no attribute '__getattr__'


In [ ]:
pip uninstall pathway

Found existing installation: pathway 0.16.3
Uninstalling pathway-0.16.3:
  Would remove:
    /usr/local/bin/pathway
    /usr/local/lib/python3.10/dist-packages/pathway-0.16.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pathway/*
Proceed (Y/n)? y
  Successfully uninstalled pathway-0.16.3


In [ ]:
pip install pathway

Y
  Using cached pathway-0.16.3-cp310-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached pathway-0.16.3-cp310-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.6 MB)
